# Hierarchical SSD demonstration

This notebook showcases the hierarchical subsystem descriptor (SSD) that
combines gate-level transitions, deduplicated gate-path nodes and qubit
subsystems.  The examples below build a descriptor for a few circuits,
inspect the automatically selected simulation methods and visualise the
resulting gate-path graph.  The final section runs a full simulation with
:class:`quasar.simulation_engine.SimulationEngine` to ensure the circuit can
still be executed end-to-end.


In [ ]:
from quasar.circuit import Circuit
from quasar.ssd import build_hierarchical_ssd
from quasar.simulation_engine import SimulationEngine

# Optional plotting helpers – the notebook works without these packages.
try:
    import matplotlib.pyplot as plt
    import networkx as nx
except ImportError:  # pragma: no cover - optional runtime dependency
    plt = nx = None


In [ ]:
def describe_path_nodes(ssd):
    print(f"SSD contains {len(ssd.partitions)} gate-path nodes
")
    for idx, part in enumerate(ssd.partitions):
        qubits = [f"{group}" for group in part.subsystems]
        deps = part.dependencies if hasattr(part, "dependencies") else ()
        print(f"Node {idx} -> backend={part.backend.name} qubits={qubits} history={part.history}")
        if deps:
            print(f"    depends on: {deps}")
        if part.entangled_with:
            print(f"    entangled with: {part.entangled_with}")
    print()


def draw_path_graph(ssd, title=None):
    if nx is None or plt is None:
        print("Install networkx and matplotlib to visualise the gate-path graph.")
        return
    graph = ssd.to_path_networkx()
    if not graph:
        print("Graph is empty.")
        return
    labels = {}
    for node, data in graph.nodes(data=True):
        history = data.get("history", ())
        if not history:
            label = "INIT"
        else:
            label = "
".join(history)
        backend = data.get("backend")
        if backend:
            label += f"
[{backend}]"
        labels[node] = label
    pos = nx.spring_layout(graph, seed=42)
    plt.figure(figsize=(8, 5))
    nx.draw_networkx(graph, pos, labels=labels, node_color="#ffcc99", node_size=1600)
    plt.title(title or "Gate-path dependency graph")
    plt.axis("off")
    plt.show()


## Example 1 – GHZ preparation

Three qubits start in the zero state, undergo a Hadamard on qubit 0 and two
controlled-NOT gates.  The first two gates operate on disjoint subsystems and
are therefore deduplicated in the path graph before entanglement merges the
qubits.


In [ ]:
ghz_gates = [
    {"gate": "H", "qubits": [0]},
    {"gate": "H", "qubits": [1]},
    {"gate": "CX", "qubits": [0, 2]},
    {"gate": "CX", "qubits": [1, 2]},
]

ghz = Circuit(ghz_gates)
ssd_ghz = ghz.ssd  # already hierarchical thanks to the circuit helper

describe_path_nodes(ssd_ghz)
draw_path_graph(ssd_ghz, title="GHZ gate-path graph")


## Example 2 – Reused single-qubit paths

This circuit applies identical gate sequences on qubits 0 and 1 before
entangling them with a controlled-Z.  The SSD collapses both single-qubit
prefixes into a single gate-path node, demonstrating reuse of identical
sequences.


In [ ]:
reused_gates = [
    {"gate": "RX", "qubits": [0], "params": {"theta": 1.5708}},
    {"gate": "RY", "qubits": [0], "params": {"theta": 0.7854}},
    {"gate": "RX", "qubits": [1], "params": {"theta": 1.5708}},
    {"gate": "RY", "qubits": [1], "params": {"theta": 0.7854}},
    {"gate": "CZ", "qubits": [0, 1]},
]

reused = Circuit(reused_gates)
ssd_reused = reused.ssd

describe_path_nodes(ssd_reused)
draw_path_graph(ssd_reused, title="Reused path graph")


## Example 3 – Full simulation

The final cell runs a dense statevector simulation.  The returned SSD comes
from the scheduler, while rebuilding the hierarchical descriptor afterwards
reveals the gate-path assignments used during analysis.


In [ ]:
engine = SimulationEngine()
sim_result = engine.simulate(ghz)
print(f"Simulation completed with {len(sim_result.ssd.partitions)} partitions.")

hierarchical_after_run = build_hierarchical_ssd(ghz)
describe_path_nodes(hierarchical_after_run)
